In [2]:
# 라이브러리 임포트
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb

In [3]:
#  날짜 기반 파생변수 생성 함수
def make_date_feats(df: pd.DataFrame) -> pd.DataFrame:
    """
    날짜 컬럼(영업일자)으로부터 다양한 달력 기반 변수 생성
    - 연, 월, 일, 요일, 주말 여부
    - 월/요일을 주기적으로 표현하는 sin, cos 값
    - 일(dayofyear) 기반 계절성 표현
    - 주차(weekofyear), 월 시작/종료 여부
    """
    df_c = df.copy()
    dt = pd.to_datetime(df_c["영업일자"])

    # 기본 날짜 정보
    df_c["year"] = dt.dt.year
    df_c["month"] = dt.dt.month
    df_c["day"] = dt.dt.day
    df_c["weekday"] = dt.dt.weekday
    df_c["is_weekend"] = df_c["weekday"].isin([5, 6]).astype(int)

    # 주기적 패턴 (sin, cos 인코딩)
    df_c["month_sin"] = np.sin(2*np.pi*df_c["month"]/12.0)
    df_c["month_cos"] = np.cos(2*np.pi*df_c["month"]/12.0)
    df_c["wday_sin"] = np.sin(2*np.pi*df_c["weekday"]/7.0)
    df_c["wday_cos"] = np.cos(2*np.pi*df_c["weekday"]/7.0)

    # 1년 중 며칠째인지
    df_c["doy"] = dt.dt.dayofyear
    df_c["doy_sin"] = np.sin(2*np.pi*df_c["doy"]/365)
    df_c["doy_cos"] = np.cos(2*np.pi*df_c["doy"]/365)

    # 주차 정보, 월 시작/끝 여부
    df_c["weekofyear"] = dt.dt.isocalendar().week.astype(int)
    df_c["is_month_start"] = dt.dt.is_month_start.astype(int)
    df_c["is_month_end"] = dt.dt.is_month_end.astype(int)

    return df_c

In [4]:
#  학습 단계에서 사용할 지연/롤링 피처
#   - 과거 매출로부터 규칙적으로 파생변수 생성
def add_train_lag_roll_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    item_id(상품)별로 정렬 후, 과거 정보로 다양한 피처를 만듭니다.
    - lag: 어제/7일전/14일전/28일전 매출
    - rolling: 최근 7/14/28일 평균/표준편차/중앙값, 최솟값/최댓값
    - zeros: 최근 N일 동안 매출이 0인 날 수
    - days_since_nz: 마지막 '0이 아닌 매출' 이후 며칠 지났는지
    - trend7: 최근 7일의 선형 추세(기울기)
    - same weekday rolling: 같은 요일 기준 최근 4회 평균
    - ratio/volatility: 비율, 변동성 지표
    """
    df = df.sort_values(["item_id","영업일자"]).copy()
    g = df.groupby("item_id")["매출수량"]

    # 1) 지연 변수(lag)
    for lag in [1, 7, 14, 28]:
        df[f"lag{lag}"] = g.shift(lag)

    # 2) 기본 롤링 통계
    df["roll7_mean"]   = g.shift(1).rolling(7).mean()
    df["roll14_mean"]  = g.shift(1).rolling(14).mean()
    df["roll7_std"]    = g.shift(1).rolling(7).std()
    df["roll7_median"] = g.shift(1).rolling(7).median()

    # 3) 28일 최솟값/최댓값
    df["min28"] = g.shift(1).rolling(28).min()
    df["max28"] = g.shift(1).rolling(28).max()

    # 4) 최근 N일간 0인 날 수
    z = (df["매출수량"] == 0).astype(int)
    gz = df.assign(zflag=z).groupby("item_id")["zflag"]
    df["zeros7"]  = gz.shift(1).rolling(7).sum()
    df["zeros14"] = gz.shift(1).rolling(14).sum()
    df["zeros28"] = gz.shift(1).rolling(28).sum()

    # 5) 마지막 '0이 아닌 매출' 이후 경과일수 (최대 60으로 클리핑)
    def _dsls(series: pd.Series) -> pd.Series:
        prev = series.shift(1).fillna(0).values
        out = np.zeros_like(prev, dtype=float)
        cnt = 0
        for i, v in enumerate(prev):
            if v > 0:
                cnt = 0
            else:
                cnt += 1
            out[i] = cnt
        out = np.clip(out, 0, 60)
        return pd.Series(out, index=series.index)
    df["days_since_nz"] = df.groupby("item_id")["매출수량"].transform(_dsls)

    # 6) 최근 7일 선형추세 기울기(상승/하강 경향)
    def _trend_7(x: pd.Series) -> float:
        if x.isna().sum() > 0:
            return np.nan
        y = x.values.astype(float)
        x_idx = np.arange(len(y))
        return np.polyfit(x_idx, y, 1)[0]
    df["trend7"] = g.shift(1).rolling(7).apply(lambda s: _trend_7(s), raw=False)

    # 7) 같은 요일 기준 최근 4회 평균
    df["weekday"] = pd.to_datetime(df["영업일자"]).dt.weekday
    grp = df.groupby(["item_id","weekday"])["매출수량"]
    df["weekday_roll4_mean"] = grp.shift(1).rolling(4).mean()

    # 8) 비율/변동성
    df["lag1_div_lag7"]   = df["lag1"] / (df["lag7"] + 1e-6)
    df["lag1_minus_lag7"] = df["lag1"] - df["lag7"]
    df["vol7"]            = df["roll7_std"] / (df["roll7_mean"] + 1e-6)

    # 9) mean28 기반 비율
    df["mean28"]             = g.shift(1).rolling(28).mean()
    df["lag1_div_mean28"]    = df["lag1"] / (df["mean28"] + 1e-6)
    df["lag7_div_mean28"]    = df["lag7"] / (df["mean28"] + 1e-6)
    df["roll7_div_mean28"]   = df["roll7_mean"] / (df["mean28"] + 1e-6)

    return df

In [5]:
#  EWM/장기 롤링/분위수 기반 피처 (학습 단계)
def add_ewm_long_quant_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    - EWM: 최근 데이터에 더 큰 가중치(변화에 민감)
    - 장기 롤링 평균: 60/90일
    - 분위수(25%, 75%)와 분위수 비율(cv28)
    + item별 평균/전체 평균으로 결측 안전 채움
    """
    df = df.sort_values(["item_id","영업일자"]).copy()
    g = df.groupby("item_id")["매출수량"]

    # EWM (최근 가중치 ↑)
    df["ewm7"]    = g.shift(1).ewm(span=7, adjust=False).mean()
    df["ewm28"]   = g.shift(1).ewm(span=28, adjust=False).mean()
    df["ewm7_std"] = g.shift(1).ewm(span=7, adjust=False).std()

    # 장기 롤링 평균
    df["roll60_mean"] = g.shift(1).rolling(60, min_periods=10).mean()
    df["roll90_mean"] = g.shift(1).rolling(90, min_periods=15).mean()

    # 분위수
    df["roll28_q25"] = g.shift(1).rolling(28, min_periods=7).quantile(0.25)
    df["roll28_q75"] = g.shift(1).rolling(28, min_periods=7).quantile(0.75)

    # 분위수 비율 기반 변동성
    df["cv28"] = df["roll28_q75"] / (df["roll28_q25"] + 1e-6)

    # 안전 채움
    add_cols = ["ewm7","ewm28","ewm7_std","roll60_mean","roll90_mean","roll28_q25","roll28_q75","cv28"]
    df = safe_fillna_by_item(df, add_cols)
    return df

In [6]:
#  안전한 결측치 채움 (item_id 단위)
#   - item별 평균 -> 전체 평균 -> 0 순으로 채움
def safe_fillna_by_item(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    out = df.copy()
    if "item_id" not in out.columns:
        raise KeyError("safe_fillna_by_item requires an 'item_id' column")
    out = out.loc[:, ~out.columns.duplicated()].copy()
    for c in cols:
        item_means = out.groupby("item_id")[c].transform(lambda s: s.fillna(s.mean()))
        out[c] = out[c].fillna(item_means)
        out[c] = out[c].fillna(out[c].mean()).fillna(0.0)
    return out

In [7]:
#  모델 입력 피처 목록 (캘린더 + 지연/롤링 + 비율/변동성 + EWM/분위수)
FEATURE_COLS = [
    # calendar
    "year","month","day","weekday","is_weekend",
    "month_sin","month_cos","wday_sin","wday_cos",
    "doy","doy_sin","doy_cos","weekofyear","is_month_start","is_month_end",
    # id
    "item_id",
    # lags
    "lag1","lag7","lag14","lag28",
    # roll stats
    "roll7_mean","roll14_mean","roll7_std","roll7_median",
    "min28","max28",
    # zeros & dsls
    "zeros7","zeros14","zeros28","days_since_nz",
    # trend & weekday mean
    "trend7","weekday_roll4_mean",
    # ratios & vol
    "lag1_div_lag7","lag1_minus_lag7","vol7",
    "lag1_div_mean28","lag7_div_mean28","roll7_div_mean28",
]
FEATURE_COLS += [
    "ewm7","ewm28","ewm7_std","roll60_mean","roll90_mean","roll28_q25","roll28_q75","cv28"
]

In [8]:
#  데이터 로드 & 기본 전처리
print("Loading train...")
train = pd.read_csv("train.csv")
train["영업일자"] = pd.to_datetime(train["영업일자"])

# 범주형 → 숫자 라벨 (상품명+메뉴명 조합을 item_id로)
le = LabelEncoder()
train["item_id"] = le.fit_transform(train["영업장명_메뉴명"])

# 캘린더 파생변수
train = make_date_feats(train)

Loading train...


In [9]:
#  IQR 기반 이상치(극단값) 완화
#   - 실제 0은 유지, 0 초과 값만 IQR으로 완화
def handle_outliers_iqr(df_group: pd.DataFrame) -> pd.DataFrame:
    """
    각 item(영업장명_메뉴명) 그룹 내에서
    - 매출수량 > 0인 값에 대해 IQR 범위 밖이면 클리핑
    """
    non_zero = df_group[df_group["매출수량"] > 0]["매출수량"]
    if len(non_zero) < 5:
        return df_group  # 표본이 너무 적으면 패스
    q1, q3 = non_zero.quantile(0.25), non_zero.quantile(0.75)
    iqr = q3 - q1
    lower, upper = max(0, q1 - 1.5*iqr), q3 + 1.5*iqr
    df_group["매출수량"] = np.clip(df_group["매출수량"], lower, upper)
    return df_group

# item 단위로 이상치 처리
train = train.groupby("영업장명_메뉴명", group_keys=False).apply(handle_outliers_iqr)

/tmp/ipython-input-901663974.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby("영업장명_메뉴명", group_keys=False).apply(handle_outliers_iqr)


In [10]:
#  학습 피처 생성 (학습 데이터 전용 함수)
train = add_train_lag_roll_features(train)
train = add_ewm_long_quant_features(train)

# 모델 입력/타깃 구성 + 결측치 제거
X_all = train[FEATURE_COLS].copy()
y_all = train["매출수량"].astype(float)

mask = X_all.notna().all(axis=1)
X_all = X_all[mask]
y_all = y_all[mask]

print(f"Train matrix: {X_all.shape}, target: {y_all.shape}")

Train matrix: (97272, 46), target: (97272,)


In [11]:
#  XGBoost 앙상블 학습
#   - 타임시리즈 스플릿으로 마지막 fold에서 조기중단 길이(best_iter) 추정
#   - 전체 데이터로 best_iter만큼 다시 학습 → 앙상블 리스트에 저장
try:
    import torch
    device = "cuda" if torch.cuda.is_available() else "cpu"
except Exception:
    device = "cpu"

tscv = TimeSeriesSplit(n_splits=5)
boosters, best_iters = [], []

for seed, max_depth in [(42, 6), (13, 8), (77, 10), (101, 6)]:
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "tree_method": "hist",
        "device": device,
        "learning_rate": 0.05,
        "subsample": 0.9,
        "colsample_bytree": 0.9,
        "max_depth": max_depth,
        "gamma": 0.0,
        "lambda": 1.0,
        "alpha": 0.0,
        "seed": seed,
    }

    # 마지막 fold를 검증셋으로 사용 (보수적 조기중단)
    last_tr_idx, last_va_idx = list(tscv.split(X_all))[-1]
    dtr = xgb.DMatrix(X_all.iloc[last_tr_idx], label=y_all.iloc[last_tr_idx])
    dva = xgb.DMatrix(X_all.iloc[last_va_idx], label=y_all.iloc[last_va_idx])

    booster = xgb.train(
        params,
        dtr,
        num_boost_round=5000,
        evals=[(dva, "val")],
        early_stopping_rounds=100,
        verbose_eval=False
    )
    best_iter = booster.best_iteration

    # 전체 데이터로 best_iter만큼 재학습 (과적합 방지)
    dall = xgb.DMatrix(X_all, label=y_all)
    booster_full = xgb.train(params, dall, num_boost_round=best_iter, verbose_eval=False)

    boosters.append(booster_full)
    best_iters.append(best_iter)

print("Ensemble trained. Best iterations per model:", best_iters)

Ensemble trained. Best iterations per model: [72, 95, 109, 144]


In [12]:
#  앙상블 예측 함수 (여러 모델 평균)
def predict_ensemble(boosters, X_pred: pd.DataFrame) -> np.ndarray:
    dpred = xgb.DMatrix(X_pred)
    preds = [bst.predict(dpred) for bst in boosters]
    return np.mean(preds, axis=0)

In [13]:
#  테스트/예측 시점의 피처 생성 (미래 누수 방지)
#   - target_date에 대해 오직 과거 정보만 써서 피처를 만든 뒤 예측
print("Loading sample & tests...")
sample = pd.read_csv("sample_submission.csv")

tests = {}
for i in range(10):
    name = f"TEST_{i:02d}"
    df = pd.read_csv(f"{name}.csv")
    df["영업일자"] = pd.to_datetime(df["영업일자"])
    tests[name] = df

Loading sample & tests...


In [14]:
def build_step_features(history: pd.DataFrame, target_date: pd.Timestamp):
    """
    target_date 하루치 예측에 필요한 피처를 만든 뒤 (frame),
    FEATURE_COLS만 추출해 X_pred로 반환합니다.
    (누수 방지: 모든 롤링/ewm/분위수는 과거 데이터만 기반)
    """
    # 1) 예측 대상 아이템 목록 프레임
    items = history["영업장명_메뉴명"].unique()
    frame = pd.DataFrame({
        "영업일자": np.repeat(target_date, len(items)),
        "영업장명_메뉴명": items
    })

    # item_id 매핑
    frame["item_id"] = (
        history.drop_duplicates("영업장명_메뉴명")
               .set_index("영업장명_메뉴명")["item_id"]
               .reindex(items).values
    )

    # 캘린더 파생
    frame = make_date_feats(frame)

    # 작업용 과거 히스토리
    temp_hist = history.copy()

    # -------------------------
    # Lag (미래 누수 방지: 날짜를 lag만큼 +)
    # -------------------------
    for lag in [1, 7, 14, 28]:
        lagged = temp_hist[["영업일자","item_id","매출수량"]].copy()
        lagged["영업일자"] = lagged["영업일자"] + pd.Timedelta(days=lag)
        frame = frame.merge(
            lagged.rename(columns={"매출수량": f"lag{lag}"}),
            on=["영업일자","item_id"], how="left"
        )

    # -------------------------
    # Rolling/집계 (과거만 사용)
    # -------------------------
    roll_base = temp_hist.sort_values(["item_id","영업일자"]).copy()
    gb = roll_base.groupby("item_id")["매출수량"]

    roll_base["roll7_mean"]   = gb.rolling(7).mean().reset_index(0, drop=True)
    roll_base["roll14_mean"]  = gb.rolling(14).mean().reset_index(0, drop=True)
    roll_base["roll7_std"]    = gb.rolling(7).std().reset_index(0, drop=True)
    roll_base["roll7_median"] = gb.rolling(7).median().reset_index(0, drop=True)
    roll_base["min28"]        = gb.rolling(28).min().reset_index(0, drop=True)
    roll_base["max28"]        = gb.rolling(28).max().reset_index(0, drop=True)
    roll_base["mean28"]       = gb.rolling(28).mean().reset_index(0, drop=True)

    # 최근 N일 0 카운트
    roll_base["zflag"] = (roll_base["매출수량"] == 0).astype(int)
    gzz = roll_base.groupby("item_id")["zflag"]
    roll_base["zeros7"]  = gzz.rolling(7).sum().reset_index(0, drop=True)
    roll_base["zeros14"] = gzz.rolling(14).sum().reset_index(0, drop=True)
    roll_base["zeros28"] = gzz.rolling(28).sum().reset_index(0, drop=True)

    # EWM/장기/분위수
    roll_base["ewm7"] = gb.apply(lambda s: s.ewm(span=7, adjust=False).mean()).reset_index(0, drop=True)
    roll_base["ewm28"] = gb.apply(lambda s: s.ewm(span=28, adjust=False).mean()).reset_index(0, drop=True)
    roll_base["ewm7_std"] = gb.apply(lambda s: s.ewm(span=7, adjust=False).std()).reset_index(0, drop=True)

    roll_base["roll60_mean"] = gb.rolling(60, min_periods=10).mean().reset_index(0, drop=True)
    roll_base["roll90_mean"] = gb.rolling(90, min_periods=15).mean().reset_index(0, drop=True)

    roll_base["roll28_q25"] = gb.rolling(28, min_periods=7).quantile(0.25).reset_index(0, drop=True)
    roll_base["roll28_q75"] = gb.rolling(28, min_periods=7).quantile(0.75).reset_index(0, drop=True)

    # 하루 당겨서(+1day) target_date와 align → 누수 방지
    roll_base["영업일자"] = roll_base["영업일자"] + pd.Timedelta(days=1)
    frame = frame.merge(
        roll_base[[
            "영업일자","item_id",
            "roll7_mean","roll14_mean","roll7_std","roll7_median",
            "min28","max28","mean28",
            "zeros7","zeros14","zeros28",
            "ewm7","ewm28","ewm7_std",
            "roll60_mean","roll90_mean",
            "roll28_q25","roll28_q75",
        ]],
        on=["영업일자","item_id"], how="left"
    )

    # 마지막 비-제로 매출 이후 경과일(최근 28일 관찰)
    def dsls_for_item(iid: int) -> float:
        h = temp_hist[temp_hist["item_id"] == iid].sort_values("영업일자").tail(28)
        h_nz = h[h["매출수량"] > 0]
        if len(h_nz) == 0:
            return 60.0
        last_nz = h_nz["영업일자"].max()
        return float(min(60, (pd.to_datetime(target_date - pd.Timedelta(days=1)) - last_nz).days))

    frame["days_since_nz"] = frame["item_id"].map(dsls_for_item)

    # trend7 (최근 7일 기울기)
    def compute_trend7_per_item(item_id):
        h = temp_hist[temp_hist["item_id"] == item_id].sort_values("영업일자")["매출수량"].values[-7:]
        if len(h) < 7 or np.isnan(h).any():
            return np.nan
        x = np.arange(7)
        return np.polyfit(x, h.astype(float), 1)[0]
    frame["trend7"] = frame["item_id"].map(lambda iid: compute_trend7_per_item(iid))

    # 같은 요일 기준 최근 4회 평균
    temp_hist["weekday"] = pd.to_datetime(temp_hist["영업일자"]).dt.weekday
    target_wday = pd.to_datetime(target_date).weekday()

    def same_weekday_last4_mean(iid):
        h = temp_hist[(temp_hist["item_id"] == iid) & (temp_hist["weekday"] == target_wday)] \
                .sort_values("영업일자")["매출수량"].tail(4)
        if len(h) == 0:
            return np.nan
        return float(h.mean())

    frame["weekday_roll4_mean"] = frame["item_id"].map(same_weekday_last4_mean)

    # 비율/변동성 + 분위수 기반 변동성(cv28)
    frame["lag1_div_lag7"]     = frame["lag1"] / (frame["lag7"] + 1e-6)
    frame["lag1_minus_lag7"]   = frame["lag1"] - frame["lag7"]
    frame["vol7"]              = frame["roll7_std"] / (frame["roll7_mean"] + 1e-6)
    frame["lag1_div_mean28"]   = frame["lag1"] / (frame["mean28"] + 1e-6)
    frame["lag7_div_mean28"]   = frame["lag7"] / (frame["mean28"] + 1e-6)
    frame["roll7_div_mean28"]  = frame["roll7_mean"] / (frame["mean28"] + 1e-6)
    frame["cv28"]              = frame["roll28_q75"] / (frame["roll28_q25"] + 1e-6)

    # 최종 입력행 구성 + 결측 안전 처리
    X_pred_full = frame[FEATURE_COLS].copy()
    X_pred_full = X_pred_full.loc[:, ~X_pred_full.columns.duplicated()].copy()
    X_pred_full = safe_fillna_by_item(X_pred_full, cols=[c for c in FEATURE_COLS if c != "item_id"])
    X_pred = X_pred_full[FEATURE_COLS]
    return X_pred, frame

In [15]:
#  7일 재귀 예측 (하루 예측 → 히스토리에 반영 → 다음날 예측)
all_preds = []

for test_name, test_df in tests.items():
    test_df = test_df.copy()
    test_df["item_id"] = le.transform(test_df["영업장명_메뉴명"])
    test_df = make_date_feats(test_df)

    # 예측 시작용 히스토리(테스트의 과거 관측)
    history = test_df.sort_values(["item_id","영업일자"]).copy()
    last_date = history["영업일자"].max()

    preds_rows = []
    current_date = last_date

    for step in range(1, 8):  # 7-step
        target_date = current_date + pd.Timedelta(days=1)

        # 1) 피처 만들기(누수 방지)
        X_pred, frame = build_step_features(history, target_date)

        # 2) 앙상블 예측
        yhat = predict_ensemble(boosters, X_pred)
        yhat = np.clip(yhat, 0, None)  # 음수 방지

        # 3) 히스토리에 예측값을 추가(재귀)
        add_hist = frame[["영업일자","item_id","영업장명_메뉴명"]].copy()
        add_hist["매출수량"] = yhat
        history = pd.concat([history, add_hist], ignore_index=True)

        # 4) 제출용 행 축적
        out_row = frame[["영업일자","영업장명_메뉴명"]].copy()
        out_row["pred"] = yhat
        out_row["영업일자"] = f"{test_name}+{step}일"
        preds_rows.append(out_row)

        current_date = target_date

    # TEST 파일 하나에 대한 wide 형태 예측
    test_pred = pd.concat(preds_rows, ignore_index=True)
    wide = test_pred.pivot(index="영업일자", columns="영업장명_메뉴명", values="pred")
    all_preds.append(wide)

In [16]:
#  반올림 후 0 → 1 치환
#   - 대회 규칙/전략에 따라 설정
all_preds = [df.round(0).replace(0, 1) for df in all_preds]

In [18]:
submission = pd.concat(all_preds)
submission = submission.reset_index().rename(columns={"index": "영업일자"})
sample = pd.read_csv("sample_submission.csv")
submission = submission[sample.columns]

submission.to_csv("EWM_submission.csv", index=False, encoding="utf-8-sig")
print(f"✅ Saved")

✅ Saved
